Run the following cell with your Azure OpenAI, endpoint URL, and deployment name

In [27]:
!python -m pip install openai keyrings.alt

In [2]:
import os
from openai import AzureOpenAI

os.environ["AZURE_OPENAI_API_KEY"]="594f098863634d7f94693bc88a650105" # TODO copy your AZURE OPENAI KEY
os.environ["AZURE_OPENAI_ENDPOINT"]="https://uksouth.api.cognitive.microsoft.com/" # TODO copy your AZURE OPENAI ENDPOINT URL
os.environ["AZURE_DEPLOYMENT_NAME"]="aifashionassistant-gpt-35-1106" # TODO copy your Azure OPENAI DEPLOYMENT NAME


client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),    
  api_version="2023-12-01-preview",
)

response = client.chat.completions.create(
    model= os.getenv("AZURE_DEPLOYMENT_NAME"),
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Does Azure OpenAI support customer managed keys?"},
        {"role": "assistant", "content": "Yes, customer managed keys are supported by Azure OpenAI."},
        {"role": "user", "content": "Do other Azure AI services support this too?"}
     ]
)

print ("\nAI Assistant:\n%s\n" % response.choices[0].message.content)

print ("AI Content safey:")
for safety_category, safety_diagnostic in response.choices[0].content_filter_results.items():
    print((safety_category, safety_diagnostic))


AI Assistant:
Yes, many Azure AI services support customer managed keys. This allows customers to have more control over the encryption keys used to protect their data and helps them meet their compliance and regulatory requirements.

AI Content safey:
('hate', {'filtered': False, 'severity': 'safe'})
('self_harm', {'filtered': False, 'severity': 'safe'})
('sexual', {'filtered': False, 'severity': 'safe'})
('violence', {'filtered': False, 'severity': 'safe'})


Helper function to make requests to Azure OpenAI

In [3]:
def get_completion(client, prompt):
    messages = [{"role": "user", "content": prompt}]
    
    response = client.chat.completions.create(
        model=os.getenv("AZURE_DEPLOYMENT_NAME"),  
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )

    return response.choices[0].message.content

## Prompting Principles
- **Principle 1: Write clear and specific instructions**
- **Principle 2: Give the model time to “think”**

### Tactics

#### Tactic 1: Use delimiters to clearly indicate distinct parts of the input
- Delimiters can be anything like: ```, """, < >, `<tag> </tag>`, `:`

In [4]:
text = f"""
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.
```{text}```
"""
response = get_completion(client, prompt)
print ("\nAI Assistant:\n%s\n" % response)



AI Assistant:
Clear and specific instructions for a model will guide it towards the desired output and reduce the chances of receiving irrelevant or incorrect responses, and longer prompts can provide more clarity and context for the model, leading to more detailed and relevant outputs.



In [5]:
prompt = f"""
Generate a list of three made-up book titles along \ 
with their authors and genres. 
Provide them in JSON format with the following keys: 
book_id, title, author, genre.
"""
response = get_completion(client, prompt)
print ("\nAI Assistant:\n%s\n" % response)


AI Assistant:
```json
{
  "books": [
    {
      "book_id": 1,
      "title": "The Midnight Garden",
      "author": "Evelyn Harper",
      "genre": "Fantasy"
    },
    {
      "book_id": 2,
      "title": "The Secret of Silver Lake",
      "author": "Nathan Black",
      "genre": "Mystery"
    },
    {
      "book_id": 3,
      "title": "Echoes of Eternity",
      "author": "Samantha Wells",
      "genre": "Science Fiction"
    }
  ]
}
```



In [6]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \ 
water boiling. While that's happening, \ 
grab a cup and put a tea bag in it. Once the water is \ 
hot enough, just pour it over the tea bag. \ 
Let it sit for a bit so the tea can steep. After a \ 
few minutes, take out the tea bag. If you \ 
like, you can add some sugar or milk to taste. \ 
And that's it! You've got yourself a delicious \ 
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""
response = get_completion(client, prompt)
print("Completion for Text 1:")
print ("\nAI Assistant:\n%s\n" % response)

Completion for Text 1:

AI Assistant:
Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Pour the hot water over the tea bag.
Step 4 - Let the tea steep for a few minutes.
Step 5 - Take out the tea bag.
Step 6 - Add sugar or milk to taste.
Step 7 - Enjoy your delicious cup of tea.



In [7]:
text_2 = f"""
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \ 
walk in the park. The flowers are blooming, and the \ 
trees are swaying gently in the breeze. People \ 
are out and about, enjoying the lovely weather. \ 
Some are having picnics, while others are playing \ 
games or simply relaxing on the grass. It's a \ 
perfect day to spend time outdoors and appreciate the \ 
beauty of nature.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""
response = get_completion(client, prompt)
print("Completion for Text 2:")
print ("\nAI Assistant:\n%s\n" % response)

Completion for Text 2:

AI Assistant:
No steps provided.



In [8]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \ 
valley flows from a modest spring; the \ 
grandest symphony originates from a single note; \ 
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(client, prompt)
print ("\nAI Assistant:\n%s\n" % response)


AI Assistant:
<grandparent>: Resilience is like the mighty oak tree that withstands the fiercest storms, bending but never breaking. It is the ability to bounce back from adversity, to find strength in the face of challenges, and to persevere in the pursuit of your goals.



In [9]:
text = f"""
In a charming village, siblings Jack and Jill set out on \ 
a quest to fetch water from a hilltop \ 
well. As they climbed, singing joyfully, misfortune \ 
struck—Jack tripped on a stone and tumbled \ 
down the hill, with Jill following suit. \ 
Though slightly battered, the pair returned home to \ 
comforting embraces. Despite the mishap, \ 
their adventurous spirits remained undimmed, and they \ 
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(client, prompt_1)
print("Completion for prompt 1:")
print ("\nAI Assistant:\n%s\n" % response)

Completion for prompt 1:

AI Assistant:
1 - Jack and Jill go on a quest to fetch water, but misfortune strikes as they both tumble down a hill, yet they return home with undimmed adventurous spirits.

2 - Jack et Jill partent en quête d'eau, mais le malheur frappe alors qu'ils dégringolent tous les deux d'une colline, mais ils rentrent chez eux avec des esprits aventureux indomptés.

3 - Jack, Jill

4 - 
{
  "french_summary": "Jack et Jill partent en quête d'eau, mais le malheur frappe alors qu'ils dégringolent tous les deux d'une colline, mais ils rentrent chez eux avec des esprits aventureux indomptés.",
  "num_names": 2
}



In [10]:
prompt_2 = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by 
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the 
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(client, prompt_2)
print("\nCompletion for prompt 2:")
print ("\nAI Assistant:\n%s\n" % response)


Completion for prompt 2:

AI Assistant:
Summary: Jack and Jill, siblings, go on a quest to fetch water from a hilltop well, but misfortune strikes as Jack trips on a stone and tumbles down the hill, with Jill following suit, yet they return home with comforting embraces and their adventurous spirits undimmed.

Translation: Jack et Jill, frère et sœur, partent en quête d'eau d'un puits au sommet d'une colline, mais le malheur frappe lorsque Jack trébuche sur une pierre et dégringole la colline, suivi par Jill, mais ils rentrent chez eux avec des étreintes réconfortantes et leurs esprits aventureux intacts.

Names: Jack, Jill

Output JSON: 
{
  "french_summary": "Jack et Jill, frère et sœur, partent en quête d'eau d'un puits au sommet d'une colline, mais le malheur frappe lorsque Jack trébuche sur une pierre et dégringole la colline, suivi par Jill, mais ils rentrent chez eux avec des étreintes réconfortantes et leurs esprits aventureux intacts.",
  "num_names": 2
}



In [11]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(client, prompt)
print ("\nAI Assistant:\n%s\n" % response)


AI Assistant:
The student's solution is correct. The total cost for the first year of operations as a function of the number of square feet is indeed 450x + 100,000.



In [12]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem including the final total. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(client, prompt)
print ("\nAI Assistant:\n%s\n" % response)


AI Assistant:
The total cost for the first year of operations is the sum of the land cost, solar panel cost, and maintenance cost.

Total cost = Land cost + Solar panel cost + Maintenance cost
Total cost = $100x + $250x + $100,000 + $10x
Total cost = $360x + $100,000

So the total cost for the first year of operations as a function of the number of square feet is 360x + 100,000.

Is the student's solution the same as actual solution just calculated:
```
Yes
```
Student grade:
```
correct
```



In [13]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie
"""
response = get_completion(client, prompt)
print ("\nAI Assistant:\n%s\n" % response)


AI Assistant:
The AeroGlide UltraSlim Smart Toothbrush by Boie is a high-tech toothbrush designed to provide a superior cleaning experience. It features an ultra-slim design that allows for easy maneuverability and access to hard-to-reach areas of the mouth. The toothbrush also comes with smart technology that tracks brushing habits and provides real-time feedback to help improve oral hygiene.

The brush head is made from soft, durable bristles that are gentle on the gums and teeth, while still effectively removing plaque and debris. The bristles are also infused with silver to help prevent bacterial growth and keep the brush clean.

The AeroGlide UltraSlim Smart Toothbrush is rechargeable and comes with a long-lasting battery, making it convenient for travel and everyday use. It also connects to a smartphone app, allowing users to track their brushing habits and receive personalized recommendations for improving their oral care routine.

Overall, the AeroGlide UltraSlim Smart Toothbr

## Try experimenting on your own!

## Product review text

In [14]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

## Sentiment (positive/negative)

In [15]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Review text: '''{lamp_review}'''
"""
response = get_completion(client, prompt)
print(response)

The sentiment of the product review is positive. The customer is satisfied with the lamp, the company's customer service, and the overall experience with the product.


## Identify types of emotions

In [16]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{lamp_review}'''
"""
response = get_completion(client, prompt)
print(response)

happy, satisfied, grateful, impressed, content


## Identify anger

In [17]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{lamp_review}'''
"""
response = get_completion(client, prompt)
print(response)

No


## Extract product and company name from customer reviews

In [18]:
prompt = f"""
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys. 
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
  
Review text: '''{lamp_review}'''
"""
response = get_completion(client, prompt)
print(response)

{
  "Item": "lamp",
  "Brand": "Lumina"
}


## Doing multiple tasks at once

In [19]:
prompt = f"""
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{lamp_review}'''
"""
response = get_completion(client, prompt)
print(response)

{
  "Sentiment": "positive",
  "Anger": false,
  "Item": "lamp",
  "Brand": "Lumina"
}


## Inferring topics

In [20]:
story = """
In a recent survey conducted by the government, 
public sector employees were asked to rate their level 
of satisfaction with the department they work at. 
The results revealed that NASA was the most popular 
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings, 
stating, "I'm not surprised that NASA came out on top. 
It's a great place to work with amazing people and 
incredible opportunities. I'm proud to be a part of 
such an innovative organization."

The results were also welcomed by NASA's management team, 
with Director Tom Johnson stating, "We are thrilled to 
hear that our employees are satisfied with their work at NASA. 
We have a talented and dedicated team who work tirelessly 
to achieve our goals, and it's fantastic to see that their 
hard work is paying off."

The survey also revealed that the 
Social Security Administration had the lowest satisfaction 
rating, with only 45% of employees indicating they were 
satisfied with their job. The government has pledged to 
address the concerns raised by employees in the survey and 
work towards improving job satisfaction across all departments.
"""

## Infer 5 topics

In [21]:
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas.

Text sample: '''{story}'''
"""
response = get_completion(client, prompt)
print(response)

1. Survey
2. Job satisfaction
3. NASA
4. Social Security Administration
5. Government pledge


## Try experimenting on your own!

## Prompt flow

### Setup your dev environement

In [22]:
!python -m pip install promptflow promptflow-tools bs4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 1.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 961.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 37.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 859.8 kB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 40.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.3/101.3 kB 2.9 MB/s eta 0:00:00
   ━━

In [23]:
!pf -v

promptflow			 1.5.0

Executable '/home/codespace/.python/current/bin/python'
Python (Linux) 3.10.13 (main, Feb  6 2024, 19:53:26) [GCC 9.4.0]


In [24]:
!pf flow init --flow ./my_chatbot --type chat

Creating flow from scratch...
Creating README.md...
Creating chat.jinja2...
Creating .promptflow folder...
Creating /workspaces/genai-techdevelop/my_chatbot/flow.dag.yaml...
Creating /workspaces/genai-techdevelop/my_chatbot/openai.yaml...
Creating /workspaces/genai-techdevelop/my_chatbot/azure_openai.yaml...
Creating /workspaces/genai-techdevelop/my_chatbot/requirements.txt...
Creating /workspaces/genai-techdevelop/my_chatbot/.gitignore...
Done. Created chat flow folder: /workspaces/genai-techdevelop/my_chatbot.
The generated chat flow is requiring a connection named open_ai_connection, please follow the steps in README.md to create if you haven't done that.
You can execute this command to test the flow, pf flow test --flow ./my_chatbot --interactive


In [28]:
!pf connection create --file ./my_chatbot/azure_openai.yaml --set api_key="594f098863634d7f94693bc88a650105" --set api_base="https://uksouth.api.cognitive.microsoft.com" --name open_ai_connection

{
    "name": "open_ai_connection",
    "module": "promptflow.connections",
    "created_date": "2024-02-22T00:47:29.638666",
    "last_modified_date": "2024-02-22T00:47:29.638666",
    "type": "azure_open_ai",
    "api_key": "******",
    "api_base": "https://uksouth.api.cognitive.microsoft.com",
    "api_type": "azure",
    "api_version": "2023-07-01-preview"
}


In [30]:
!pf flow test --flow ./my_chatbot --inputs question="show"

2024-02-22 00:48:30 +0000    6730 execution.flow     INFO     Start executing nodes in thread pool mode.
2024-02-22 00:48:30 +0000    6730 execution.flow     INFO     Start to run 1 nodes with concurrency level 16.
2024-02-22 00:48:30 +0000    6730 execution.flow     INFO     Executing node chat. node run id: 043ee812-e566-48e1-8f7a-cb040117f409_chat_0
2024-02-22 00:48:30 +0000    6730 execution.flow     INFO     Node chat completes.
{
    "answer": "I'm happy to help! What would you like to see or learn about?"
}


In [32]:
!pf flow test --flow ./part2/chat_with_wikipedia --inputs question="Who is running for US president 2024?"

2024-02-22 00:49:02 +0000    6976 execution.flow     INFO     Start executing nodes in thread pool mode.
2024-02-22 00:49:02 +0000    6976 execution.flow     INFO     Start to run 5 nodes with concurrency level 16.
2024-02-22 00:49:02 +0000    6976 execution.flow     INFO     Executing node extract_query_from_question. node run id: 424108c3-84de-4449-8c87-4b8f8515c305_extract_query_from_question_0
2024-02-22 00:49:02 +0000    6976 execution.flow     INFO     Node extract_query_from_question completes.
2024-02-22 00:49:02 +0000    6976 execution.flow     INFO     Executing node get_wiki_url. node run id: 424108c3-84de-4449-8c87-4b8f8515c305_get_wiki_url_0
2024-02-22 00:49:03 +0000    6976 execution.flow     INFO     [get_wiki_url in line 0 (index starts from 0)] stdout> Could not find US presidential candidates 2024. Similar entity: ['2024 Democratic Party presidential candidates', '2024 Republican Party presidential primaries'].
2024-02-22 00:49:03 +0000    6976 execution.flow     INFO

In [33]:
!pf flow test --flow ./part2/functions --inputs question="What is the weather in Bristol?"

2024-02-22 00:49:23 +0000    7139 execution.flow     INFO     Start executing nodes in thread pool mode.
2024-02-22 00:49:23 +0000    7139 execution.flow     INFO     Start to run 2 nodes with concurrency level 16.
2024-02-22 00:49:23 +0000    7139 execution.flow     INFO     Executing node use_functions_with_chat_models. node run id: 86352121-f7bc-4a16-9d7b-a411879d9033_use_functions_with_chat_models_0
2024-02-22 00:49:24 +0000    7139 execution.flow     INFO     Node use_functions_with_chat_models completes.
2024-02-22 00:49:24 +0000    7139 execution.flow     INFO     Executing node run_function. node run id: 86352121-f7bc-4a16-9d7b-a411879d9033_run_function_0
2024-02-22 00:49:24 +0000    7139 execution.flow     INFO     [run_function in line 0 (index starts from 0)] stdout> get_current_weather
2024-02-22 00:49:24 +0000    7139 execution.flow     INFO     [run_function in line 0 (index starts from 0)] stdout> {'location': 'Bristol'}
2024-02-22 00:49:24 +0000    7139 execution.flow  